In [11]:
import os
import hashlib

In [12]:
# First we need secure source of randomness
# valid entropy bit sizes [128, 160, 192, 224, 256]
entropyBitSize = 128
# secure source of randomness
entropyBytes = os.urandom(entropyBitSize // 8) # Byte has 8 bits
print("Entropy:", entropyBytes.hex())

Entropy: 604f02a726623a819b5af7e823d3b24c


In [13]:
# we convert our entropy bytes to bit array so its easier to work with
from bitarray import bitarray
entropyBits = bitarray()
entropyBits.frombytes(entropyBytes)
print("Entropy bits:", entropyBits)

Entropy bits: bitarray('01100000010011110000001010100111001001100110001000111010100000011001101101011010111101111110100000100011110100111011001001001100')


In [14]:
# we need to add checksum bits at the of our are entropy
checksumLen = entropyBitSize // 32 # checksum length depends on the length of entropy
print("checksum size:", checksumLen)

# We need to take a hash of our entropy

hashBytes = hashlib.sha256(entropyBytes).digest()
print("hashed bytes:", hashBytes.hex())

hashBits = bitarray()
hashBits.frombytes(hashBytes)
checksum = hashBits[:checksumLen]
print("checksum bits:", checksum)

checksum size: 4
hashed bytes: cd15880675857f3e0db3348d1e85304a7e2a0ec97e64abea64bd300aae3dd16b
checksum bits: bitarray('1100')


In [15]:
# Add checksum bits at the end of entropy
entropyBits.extend(checksum)
print("entropy length:", len(entropyBits))

entropy length: 132


In [16]:
# Get indexed from bits
indexes = list()
from bitarray.util import ba2int

for idx in range(len(entropyBits) // 11):
    startIdx = idx * 11
    endIdx = startIdx + 11
    wordIndex = ba2int(entropyBits[startIdx:endIdx])
    indexes.append(wordIndex)

print(indexes)

[770, 960, 1358, 614, 285, 518, 875, 759, 1857, 244, 1892, 1228]


In [17]:
# Load bip 39 words
fileObj = open("bip-0039/english.txt", "r")
words = fileObj.read().splitlines()
fileObj.close()
print("words len:", len(words))

words len: 2048


In [18]:
# Map indexes onto words
mnemonic = list(map(lambda idx: words[idx] , indexes))
print("mnemonic:", mnemonic)

mnemonic: ['gate', 'job', 'prefer', 'error', 'casual', 'dolphin', 'hope', 'galaxy', 'trial', 'burden', 'uncle', 'office']


In [19]:
# Generate salt
password = ""
salt = "mnemonic" + password

In [20]:
# Finally, we derive seed

mnemonicStr = ' '.join(mnemonic)
seed = hashlib.pbkdf2_hmac(
    "sha512",
    mnemonicStr.encode("utf-8"),
    salt.encode("utf-8"),
    2048
)

print("seed len:", len(seed))
print("seed hex:", seed.hex())
print("priv key:", seed[0:32].hex())
print("chain co:", seed[32:64].hex())

seed len: 64
seed hex: 29eab2e74c0d72196aa2394a84fe29ad6feb16951a6fe77e936677de0ba726ab77a56e16cf3f9ae368d9a097d7c72acd7239c3f5b0dd844b711e7eef4d3e57c7
priv key: 29eab2e74c0d72196aa2394a84fe29ad6feb16951a6fe77e936677de0ba726ab
chain co: 77a56e16cf3f9ae368d9a097d7c72acd7239c3f5b0dd844b711e7eef4d3e57c7
